# 📘 DevOps 업무전환 - Epic 생성
- **목적**: DevOps 전환을 위해 작성한 항목들을 에픽과 스토리로 구분하고, 에픽의 우선순위를 산정
- **입력 데이터**: https://www.notion.so/devops-77d2cc12fdb74c298e98cc8a45d812db?source=copy_link
- **주요 기능**
  - 항목들 에픽 구성(합치거나, 스토리로 만들거나, 합쳐서 하나의 에픽으로 만들거나)
  - 에픽 우선 순위 산정
- **작성자**: 김동영
- **작성일**: 2025-08-01

# 0. 환경 설정

In [3]:
!pip install "langchain>=0.3.25,<0.4.0" "fastapi[standard]>=0.100.0" "pydantic>=2.0.0" "openai>=1.84.0,<2.0.0" "python-dotenv>=1.1.0,<2.0.0" "langchain-community>=0.3.24,<0.4.0" "pandas>=2.0.0,<3.0.0" "langgraph>=0.5.2,<0.6.0"


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [5]:
!pip list |grep langchain

langchain                                0.3.27
langchain-community                      0.3.27
langchain-core                           0.3.74
langchain-text-splitters                 0.3.9


In [1]:
import os, sys
sys.path.append(os.path.abspath("../src"))

import dotenv
import json

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

from epic.services import EpicGeneratorAgent
# from epic.prompts import TASK_TO_EPIC_CONVERTER_PROMPT

In [2]:
dotenv.load_dotenv('../.env')

True

# 1. 데이터 정제

In [3]:
# 에픽 생성 llm 모델 생성
epic_agent = EpicGeneratorAgent(
    openai_api_key = os.getenv("OPENAI_API_KEY")
)

/Users/junkiwon/Desktop/IOL/work directory/project_manage_agent/src/epic/services.py:19: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(


In [4]:
# 프롬프트 재수정
TASK_TO_EPIC_CONVERTER_PROMPT = ChatPromptTemplate.from_messages(
    [SystemMessagePromptTemplate.from_template(
        """
        [Role] 너는 숙련된 프로젝트 매니저이자 업무 분석 전문가다.
        [Goal] 정리되지 않은 업무 리스트를 분석하여 관련 업무들을 그룹핑하고, 각 그룹을 하나의 Epic으로 변환하는 것.
        [Instruction] 
          1. 주어진 업무 리스트를 분석하여 기능적으로 관련된 업무들을 식별한다.
          2. 유사하거나 같은 기능 영역의 업무들을 하나의 Epic으로 그룹핑한다.
          3. 각 Epic에 포함되지 않은 개별 업무는 별도의 Epic으로 생성한다.
          4. Epic 제목은 해당 기능 영역을 대표하는 명확한 이름으로 작성한다.
        [Constraints]
          - 출력은 반드시 [Output Format]을 따라야 한다.
          - Epic은 하나 이상의 스프린트에 걸쳐 진행되는 큰 기능 단위여야 한다.
          - 불필요한 설명, 주석, 마크다운 없이 JSON만 출력한다.
          - 최대 {max_epics}개의 Epic을 생성한다.
          - 각 업무들로 에픽을 구성할 때 최소한의 에픽으로 구성한다
          - acceptance_criteria는 그룹핑된 업무들을 기반으로 구체적이고 측정 가능한 기준으로 작성한다.
          - 개발자의 업무 영역만 작성한다.
          - 각 Epic의 included_tasks에는 해당 Epic에 포함된 원본 업무들을 나열한다.
        [Output Format]
        [
          {{
            "title": "에픽 제목",
            "description": "에픽 설명", 
            "business_value": "비즈니스 가치",
            "priority": "High|Medium|Low",
            "acceptance_criteria": ["수락 기준 1", "수락 기준 2", "수락 기준 3"],
            "included_tasks": ["포함된 원본 업무 1", "포함된 원본 업무 2"],
          }}
        ]
        """
    ),

    HumanMessagePromptTemplate.from_template(
        """
        [Input] 정리되지 않은 업무 리스트:
        {user_input}
        
        [Context] {project_info}
        """
    )
    ]
)

In [13]:
# llm prompt에 포함할 유저 지시내용 작성(프로세스의 업무사항)
# - 입력받은 업무를 에픽으로 변환, 여러개를 하나의 에픽으로 구성하고 구성항목을 하위항목(스토리)으로 두거나, 하나의 업무를 하나의 에픽으로 구성
user_input = """
- GitHub Actions 또는 Jenkins를 활용한 End-to-End 자동화 구성
    - PR 생성 시 Lint, Unit Test, Build, Code Quality Check 자동 실행
    - Dockerfile 실행 오류 및 종속성 검증 포함
- PR 병합은 `main`, `develop` 브랜치로만 제한 (브랜치 보호 규칙 설정)
- PR 템플릿 제공 및 코드 리뷰 프로세스 문서화
- 각 브랜치/환경별 `.env`, Secrets Manager, Parameter Store 등 사용
- CodeBuild 연동 (`buildspec.yml`)
- DockerHub 또는 ECR에 결과 이미지 자동 푸시
- 결과 Slack/이메일/웹훅 알림 연동
- 배포 트리거: GitHub Webhook, PR 병합, 수동 승인
- **배포 전략 선택**
    - 블루그린 배포 (Blue/Green)
    - 카나리아 배포
    - 롤링 배포
- AWS IAM Role, SSH 키를 통한 보안 배포
- 배포 전후 헬스체크 및 자동 롤백 구성
- 배포 이력/승인 기록 관리
- **정책 생성**
    - 파이프라인/워크플로 명명 규칙 (`ci-<service>`, `deploy-<env>`)
    - 신규 프로젝트 기본 템플릿 및 체크리스트 제공
    - 배포 승인 프로세스 및 롤백 기준 명시
"""

In [14]:
generated_epics_str = epic_agent._generate_epics_with_llm(
    prompt = TASK_TO_EPIC_CONVERTER_PROMPT,
    user_input = user_input,
    project_info= '',
    max_epics=10
)

INFO:epic.services:에픽 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:epic.services:에픽 생성 완료


In [28]:
parsed = json.loads(generated_epics_str)
parsed 

[{'title': 'CI/CD 자동화 구성',
  'description': 'GitHub Actions 또는 Jenkins를 활용하여 End-to-End 자동화 구성 및 배포 전략 수립',
  'business_value': '자동화된 CI/CD 프로세스를 통해 개발 효율성을 높이고 배포 오류를 줄인다.',
  'priority': 'High',
  'acceptance_criteria': ['PR 생성 시 Lint, Unit Test, Build, Code Quality Check가 자동 실행된다.',
   'Dockerfile 실행 오류 및 종속성 검증이 포함된다.',
   '결과 이미지가 DockerHub 또는 ECR에 자동으로 푸시된다.'],
  'included_tasks': ['GitHub Actions 또는 Jenkins를 활용한 End-to-End 자동화 구성',
   'PR 생성 시 Lint, Unit Test, Build, Code Quality Check 자동 실행',
   'Dockerfile 실행 오류 및 종속성 검증 포함',
   'CodeBuild 연동 (`buildspec.yml`)',
   'DockerHub 또는 ECR에 결과 이미지 자동 푸시']},
 {'title': '브랜치 보호 및 PR 관리',
  'description': '브랜치 보호 규칙 설정 및 PR 템플릿 제공을 통한 코드 품질 관리',
  'business_value': '코드 품질을 높이고 협업을 원활하게 한다.',
  'priority': 'Medium',
  'acceptance_criteria': ['PR 병합은 `main`, `develop` 브랜치로만 제한된다.',
   'PR 템플릿이 제공된다.',
   '코드 리뷰 프로세스가 문서화된다.'],
  'included_tasks': ['PR 병합은 `main`, `develop` 브랜치로만 제한 (브랜치 보호 규칙 설정)',
   'PR 템플릿 제공 및 코드 리뷰 프로세스 문서화']},
 {'ti

In [30]:
management_infra_user_input = """
- EC2, ECS, EKS, S3, RDS, ALB 등 기존 사용 중인 AWS 리소스를 관리하고 주기적으로 미사용 리소스를 정리하고 분석하여 비용 최적화 방안을 모색
- 환경별 구성 분리, 리소스 태깅
- Auto Scaling, 예약 인스턴스, spot 인스턴스, 인스턴스 사양 관리 등 비용 최적화 방안을 찾음
- 인증서 자동 갱신 (ACM, certbot 등)
- IAM Role 기반 최소 권한 접근 제어를 하도록 구성하고 기존 IAM role, 정책, 사용자를 정리하고 관리
- Route53, 도메인, SSL 인증서 갱신 관리
- 파트너사와의 역할 분리 및 접근 제어 정책 관리 (접속 권한, 키 회수 등)
- 리소스 태깅 정책 적용 및 예산 분류 가능하게 구성
- **정책 생성 (**인프라 리소스 네이밍 룰 정의, 키 회수, MFA, 보안 그룹 등 보안 정책 명시, 주기적 비용 리포트 및 미사용 자원 정리 기준,민감정보 관리 기준 정책 등)
"""

In [31]:
generated_epics_infra = epic_agent._generate_epics_with_llm(
    prompt = TASK_TO_EPIC_CONVERTER_PROMPT,
    user_input = management_infra_user_input,
    project_info= '',
    max_epics=10
)

INFO:epic.services:에픽 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:epic.services:에픽 생성 완료


In [32]:
parsed_infra = json.loads(generated_epics_infra)

for x in parsed_infra:
    print(x)

{'title': 'AWS 리소스 관리 및 비용 최적화', 'description': '기존 사용 중인 AWS 리소스를 관리하고 비용 최적화 방안을 모색하는 작업', 'business_value': '비용 절감 및 리소스 효율성 향상', 'priority': 'High', 'acceptance_criteria': ['모든 미사용 AWS 리소스가 식별되고 정리되었음', '비용 최적화 방안이 문서화되어 있음', '리소스 태깅 정책이 적용되어 예산 분류가 가능함'], 'included_tasks': ['EC2, ECS, EKS, S3, RDS, ALB 등 기존 사용 중인 AWS 리소스를 관리하고 주기적으로 미사용 리소스를 정리하고 분석하여 비용 최적화 방안을 모색', '환경별 구성 분리, 리소스 태깅', 'Auto Scaling, 예약 인스턴스, spot 인스턴스, 인스턴스 사양 관리 등 비용 최적화 방안을 찾음', '리소스 태깅 정책 적용 및 예산 분류 가능하게 구성']}
{'title': '인증서 관리 및 자동 갱신', 'description': 'SSL 인증서 및 도메인 관리와 관련된 업무', 'business_value': '서비스의 보안성 강화 및 운영 효율성 증대', 'priority': 'Medium', 'acceptance_criteria': ['모든 SSL 인증서가 자동으로 갱신되도록 설정됨', '도메인 관리 프로세스가 문서화되어 있음'], 'included_tasks': ['인증서 자동 갱신 (ACM, certbot 등)', 'Route53, 도메인, SSL 인증서 갱신 관리']}
{'title': 'IAM 및 접근 제어 관리', 'description': 'IAM Role 기반의 최소 권한 접근 제어 및 정책 관리', 'business_value': '보안 강화 및 접근 관리 효율성 증대', 'priority': 'High', 'acceptance_criteria': ['모든 IAM role, 정책, 사용자가 정리되고 관리됨', '접근 제어 정책

In [5]:
monitoring_and_log_user_input = """
- Prometheus + Grafana로 메트릭 대시보드 구성
- Loki + Grafana로 서비스 중인 서버의 로그 수집 및 시각화
- 주요 이벤트(에러, 로그인, 배포 등) 자동 수집 및 보관 방법 모색 및 적용
- Slack으로 알림연동
- 로그 스키마/레벨 통일, JSON 기반 구조로 통일
- 개인정보/보안 정보 마스킹 처리 방안 통일
- 로그 보존 주기 및 저장 정책 수립
- Datadog, ELK, CloudWatch Logs 등과 연동하는 방안 모색
- **정책 생성(**서비스별 로그 레벨 및 포맷 표준화, 알람 임계치 설정 기준 문서화, 로그 보관 주기 및 백업 연계 정의 등)
"""

generated_epics_infra = epic_agent._generate_epics_with_llm(
    prompt = TASK_TO_EPIC_CONVERTER_PROMPT,
    user_input = monitoring_and_log_user_input,
    project_info= '',
    max_epics=10
)

parsed_infra = json.loads(generated_epics_infra)



INFO:epic.services:에픽 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:epic.services:에픽 생성 완료


In [7]:
container_user_input = """
- Dockerfile 멀티스테이지 빌드 구성
- Docker Compose로 환경별 서비스 분리
- 이미지 자동 배포방식 모색 및 설정
- 환경변수 관리 방식 설정( `.env`, Secrets Manager, ConfigMap 등)
- 컨테이너 네트워크 브릿지, 포트 바인딩, DNS 구성
- 볼륨 마운트 및 데이터 영속화 정책
- **Kubernetes 적용 여부 판단 후 적용 또는 미적용(**`eksctl`, `Helm` 사용한 클러스터 구성, Deployment, Service, Ingress 등 리소스 정의, Helm Chart로 템플릿화, HPA (Horizontal Pod Autoscaler) 적용 등)
- **정책 생성 (**배포 리소스 네이밍 가이드 (Helm Chart, k8s 리소스 등), Secret, ConfigMap 분리 저장 및 암호화 기준, 이미지 취약점 검사 및 업데이트 주기 문서화 등)
"""

generated_epics_container = epic_agent._generate_epics_with_llm(
    prompt = TASK_TO_EPIC_CONVERTER_PROMPT,
    user_input = container_user_input,
    project_info= '',
    max_epics=10
)

parsed_container = json.loads(generated_epics_container)

INFO:epic.services:에픽 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:epic.services:에픽 생성 완료


In [9]:
parsed_container

[{'title': 'Docker 환경 설정 및 관리',
  'description': 'Docker 환경을 설정하고 관리하기 위한 작업들을 포함합니다.',
  'business_value': '효율적인 개발 환경 구축 및 운영',
  'priority': 'High',
  'acceptance_criteria': ['Dockerfile 멀티스테이지 빌드가 성공적으로 구성되어야 한다.',
   'Docker Compose를 통해 환경별 서비스가 분리되어야 한다.',
   '환경변수 관리 방식이 설정되어야 한다.'],
  'included_tasks': ['Dockerfile 멀티스테이지 빌드 구성',
   'Docker Compose로 환경별 서비스 분리',
   '환경변수 관리 방식 설정( `.env`, Secrets Manager, ConfigMap 등)']},
 {'title': '컨테이너 네트워크 및 데이터 관리',
  'description': '컨테이너의 네트워크 및 데이터 영속성을 관리하는 작업들을 포함합니다.',
  'business_value': '안정적인 데이터 관리 및 네트워크 구성',
  'priority': 'Medium',
  'acceptance_criteria': ['컨테이너 네트워크가 브릿지, 포트 바인딩, DNS 구성이 완료되어야 한다.',
   '볼륨 마운트 및 데이터 영속화 정책이 수립되어야 한다.'],
  'included_tasks': ['컨테이너 네트워크 브릿지, 포트 바인딩, DNS 구성', '볼륨 마운트 및 데이터 영속화 정책']},
 {'title': 'Kubernetes 클러스터 구성 및 관리',
  'description': 'Kubernetes 클러스터를 구성하고 관리하기 위한 작업들을 포함합니다.',
  'business_value': '확장성과 관리 용이성을 제공하는 클라우드 네이티브 환경 구축',
  'priority': 'High',
  'acceptance_criteria': ['Kubernetes 

In [10]:
iac_user_input = """
- 기존 AWS 리소스를 포함하여 인프라를 선언형 코드로 관리하도록 전환(VPC, EC2, RDS, ALB, S3, IAM 등 리소스를 Terraform 또는 AWS CDK로 점진적 코드화, 향후 멀티 클라우드 및 온프레미스 리소스 통합 관리 고려)
- 모듈화 및 환경별 Workspace 분리 적용 (dev, staging, prod) 및 디렉토리 구조/변수/출력 표준화 가이드 문서화
- 코드 리뷰 및 PR 기반 인프라 변경 승인 프로세스 구축 (PR 시 terraform fmt, terraform validate, 보안 스캔(tfsec, checkov) 자동 실행)
- CI/CD 연동으로 IaC 자동 적용 (GitHub Actions 또는 Jenkins 사용)
- PR 생성 시 terraform plan 결과 리뷰 → 테스트 환경 자동 배포(Sandbox) → smoke test → 승인 후 apply 진행
- IaC 상태 백엔드(S3 + DynamoDB) 구성하여 협업 충돌 방지, State 백업 및 복구 절차, Lock 해제 절차 문서화
- 보안 정책 내재화 (IAM Role 및 Policy 정의를 코드로 관리, Security Group, NACL, S3 정책 등 보안 설정 자동화, 민감 정보는 AWS Secrets Manager / SSM Parameter Store로 분리, Secret 자동 Rotate 정책 적용, 환경별 Secret 분리)
- 운영 최적화 구성 (Auto Scaling, Spot Instance, 예약 인스턴스 관리 포함, 모듈/변수/출력 구조를 문서화하여 재사용성 확보, 리소스 태그 관리(project, owner, env 등) 및 비용 분석 연계, AWS Compute Optimizer/Cost Explorer 연동)
- 정책 생성 (IaC 리소스 명명 규칙(project-env-module), 접근 권한 분리: 개발자/운영자/관리자 Role 정의, 정기적인 모듈 업데이트 및 리팩토링 주기 정의, Terraform 코드 린트 + 포맷 + Static Analysis 적용)
- 운영 모니터링 자동 구성 (CloudWatch 대시보드, 알람, 로그 그룹을 IaC 코드로 자동 생성하여 배포 직후 모니터링 환경이 완비되도록 구성)
"""

generated_epics_iac = epic_agent._generate_epics_with_llm(
    prompt = TASK_TO_EPIC_CONVERTER_PROMPT,
    user_input = iac_user_input,
    project_info= '',
    max_epics=10
)

parsed_iac = json.loads(generated_epics_iac)

INFO:epic.services:에픽 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:epic.services:에픽 생성 완료


In [11]:
parsed_iac

[{'title': '인프라 관리 자동화',
  'description': 'AWS 리소스를 Terraform 또는 AWS CDK로 관리하고, CI/CD 연동을 통해 인프라 변경을 자동화하는 Epic입니다.',
  'business_value': '인프라 관리의 효율성을 높이고, 변경 사항의 안정성을 확보합니다.',
  'priority': 'High',
  'acceptance_criteria': ['기존 AWS 리소스를 Terraform 또는 AWS CDK로 코드화 완료',
   'CI/CD 연동을 통해 IaC 자동 적용 프로세스 구축',
   'PR 생성 시 terraform plan 결과 리뷰 및 테스트 환경 자동 배포 기능 구현'],
  'included_tasks': ['기존 AWS 리소스를 포함하여 인프라를 선언형 코드로 관리하도록 전환',
   'CI/CD 연동으로 IaC 자동 적용',
   'PR 생성 시 terraform plan 결과 리뷰 → 테스트 환경 자동 배포(Sandbox) → smoke test → 승인 후 apply 진행']},
 {'title': '모듈화 및 환경 관리',
  'description': '인프라 코드의 모듈화 및 환경별 Workspace 분리를 통해 관리의 일관성을 높이는 Epic입니다.',
  'business_value': '환경별 관리의 용이성을 제공하고, 코드의 재사용성을 높입니다.',
  'priority': 'Medium',
  'acceptance_criteria': ['모듈화 및 환경별 Workspace 분리 적용 완료',
   '디렉토리 구조/변수/출력 표준화 가이드 문서화 완료'],
  'included_tasks': ['모듈화 및 환경별 Workspace 분리 적용 (dev, staging, prod) 및 디렉토리 구조/변수/출력 표준화 가이드 문서화']},
 {'title': '보안 정책 및 관리',
  'description': '인프라의 보안을 강화하기 위한 정책을 정의하고 자동화하는 E

In [22]:
parsed_1 = json.loads(generated_epics_str_1)
print(parsed_1)

[{'title': 'CI/CD 자동화 구성', 'description': 'GitHub Actions 또는 Jenkins를 활용하여 End-to-End 자동화 구성 및 배포 전략 수립', 'business_value': '개발 프로세스의 효율성을 높이고, 배포 오류를 줄여 신속한 피드백을 제공', 'priority': 'High', 'acceptance_criteria': ['End-to-End 자동화가 성공적으로 구성되어 PR 생성 시 Lint, Unit Test, Build, Code Quality Check가 자동 실행됨', 'Dockerfile 실행 오류 및 종속성 검증이 성공적으로 수행됨', '결과 이미지가 DockerHub 또는 ECR에 자동으로 푸시됨'], 'included_tasks': ['GitHub Actions 또는 Jenkins를 활용한 End-to-End 자동화 구성(PR 생성 시 Lint, Unit Test, Build, Code Quality Check 자동 실행, Dockerfile 실행 오류 및 종속성 검증 포함)', 'DockerHub 또는 ECR에 결과 이미지 자동 푸시']}, {'title': '브랜치 보호 및 관리', 'description': 'PR 병합을 위한 브랜치 보호 규칙 설정 및 환경별 비밀 관리', 'business_value': '코드 품질을 유지하고, 배포 안정성을 높임', 'priority': 'Medium', 'acceptance_criteria': ['PR 병합이 `main`, `develop` 브랜치로만 제한됨', '각 브랜치/환경별 `.env`, Secrets Manager, Parameter Store 등이 성공적으로 설정됨'], 'included_tasks': ['PR 병합은 `main`, `develop` 브랜치로만 제한 (브랜치 보호 규칙 설정)', '각 브랜치/환경별 `.env`, Secrets Manager, Parameter Store 등 사용']}, {'title': '배포 전략 및

In [23]:
for x in parsed:
    print(x['title'])

AWS 리소스 관리 및 최적화
S3 버킷 관리 및 데이터 보존
보안 및 접근 제어 정책 관리
모니터링 및 알람 설정
비용 관리 및 리포트 작성
도메인 및 SSL 인증서 관리


In [26]:
for x in parsed_infra:
    print(x['title'])

AWS 리소스 관리 및 최적화
S3 버킷 관리 및 데이터 보존
보안 및 접근 제어 정책 관리
모니터링 및 알람 설정
비용 관리 및 리포트 작성
도메인 및 SSL 인증서 관리


In [24]:
for x in parsed_1:
    print(x['title'])

NameError: name 'parsed_1' is not defined

In [26]:
# 하위 항목 구성 수정 
# *및 띄어쓰기 삭제
user_input_2 = """GitHub Actions 또는 Jenkins를 활용한 End-to-End 자동화 구성(PR 생성 시 Lint, Unit Test, Build, Code Quality Check 자동 실행, Dockerfile 실행 오류 및 종속성 검증 포함), PR 병합은 main, develop 브랜치로만 제한 (브랜치 보호 규칙 설정), 각 브랜치/환경별 .env, Secrets Manager, Parameter Store 등 사용, CodeBuild 연동 (buildspec.yml), DockerHub 또는 ECR에 결과 이미지 자동 푸시, CI/CD 결과를 Slack/이메일/웹훅 알림으로 연동, 배포 트리거 전략 세운 후 적용(GitHub Webhook, PR 병합, 수동 승인 등), 배포 전략 선택(블루그린 배포(Blue/Green), 카나리아 배포, 롤링 배포 중에 선택 후 적용), AWS IAM Role, SSH 키를 통한 보안 배포, 배포 전후 헬스체크 및 자동 롤백 구성, 배포 이력/승인 기록 관리, 정책 생성(파이프라인/워크플로 명명 규칙, 신규 프로젝트 기본 템플릿 및 체크리스트 제공, 배포 승인 프로세스 및 롤백 기준 명시, PR 템플릿 제공 및 코드 리뷰 프로세스 문서화 등)"""

In [27]:
generated_epics_str_2 = epic_agent._generate_epics_with_llm(
    prompt = TASK_TO_EPIC_CONVERTER_PROMPT,
    user_input = user_input_2,
    project_info= '',
    max_epics=10
)

INFO:epic.services:에픽 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:epic.services:에픽 생성 완료


In [28]:
parsed_2 = json.loads(generated_epics_str_1)
print(parsed_2)

[{'title': 'CI/CD 자동화 구성', 'description': 'GitHub Actions 또는 Jenkins를 활용하여 End-to-End CI/CD 자동화 프로세스를 구성한다.', 'business_value': '개발 및 배포 프로세스의 효율성을 높이고, 코드 품질을 유지하여 빠른 피드백을 제공한다.', 'priority': 'High', 'acceptance_criteria': ['PR 생성 시 Lint, Unit Test, Build, Code Quality Check가 자동으로 실행된다.', 'Dockerfile 실행 오류 및 종속성 검증이 포함된다.', 'CI/CD 결과가 Slack/이메일/웹훅으로 알림된다.'], 'included_tasks': ['GitHub Actions 또는 Jenkins를 활용한 End-to-End 자동화 구성', 'PR 생성 시 Lint, Unit Test, Build, Code Quality Check 자동 실행', 'Dockerfile 실행 오류 및 종속성 검증 포함', 'CI/CD 결과를 Slack/이메일/웹훅 알림으로 연동']}, {'title': '브랜치 보호 및 관리', 'description': '코드 품질을 보장하기 위해 특정 브랜치에 대한 보호 규칙을 설정하고 관리한다.', 'business_value': '코드의 안정성을 높이고, 팀원 간의 협업을 원활하게 한다.', 'priority': 'Medium', 'acceptance_criteria': ['PR 병합은 main, develop 브랜치로만 제한된다.', '브랜치 보호 규칙이 설정된다.'], 'included_tasks': ['PR 병합은 main, develop 브랜치로만 제한', '브랜치 보호 규칙 설정']}, {'title': '환경 설정 및 비밀 관리', 'description': '각 브랜치 및 환경에 필요한 설정을 관리하고 비밀 정보를 안전하게 저장한다.', 'business_value': '환경별 설정을 체계적으로 관리하여